---
title: "09-RR值与OR值的计算"
author: "Simon Zhou"
date: "2025-05-06"
format: 
    html:
        code-fold: false
        fig_caption: true
        number-sections: true
        toc: true
        toc-depth: 2
---

In [2]:
import stata_setup
stata_setup.config('C:/Program Files/Stata18', 'mp', splash=False)

# RR值与OR值

## RR值（Relative Risk）

## 数据导入

此章节使用网络数据，`csxmpl`

In [8]:
%%stata
webuse csxmpl, clear
list


. webuse csxmpl, clear



. list

     +------------------+
     | case   exp   pop |
     |------------------|
  1. |    1     1     7 |
  2. |    1     0    12 |
  3. |    0     1     9 |
  4. |    0     0     2 |
     +------------------+

. 


In [9]:
%%stata
cs case exp [fweight = pop]


                 |        Exposed         |
                 |   Exposed   Unexposed  |      Total
-----------------+------------------------+-----------
           Cases |         7          12  |         19
        Noncases |         9           2  |         11
-----------------+------------------------+-----------
           Total |        16          14  |         30
                 |                        |
            Risk |     .4375    .8571429  |   .6333333
                 |                        |
                 |      Point estimate    |    [95% conf. interval]
                 |------------------------+------------------------
 Risk difference |        -.4196429       |   -.7240828   -.1152029 
      Risk ratio |         .5104167       |    .2814332    .9257086 
 Prev. frac. ex. |         .4895833       |    .0742914    .7185668 
 Prev. frac. pop |         .2611111       |
                 +-------------------------------------------------
                           

### RR 值计算

```stata
cs var_case var_exp [if] [in] [weight] [,cs_options]
```

```stata
csi #a #b #c #d [,csi_options]
```

计算 RR 值使用 `cs` 命令，它是 `cohort study` 的缩写

上面 `[fweight = pop]` 中使用 `pop` 进行频数加权 `fweight`

`[,cs_options]` 可以修改置信区间等

## OR值（Odds Ratio）

### RR值与OR值的区别

**RR值**：Cohort study或者RCT中，研究者前瞻性地观察“暴露组”和“非暴露组”的发病情况，之后通过RR来评价“暴露组”研究对象的发病风险是“非暴露组”研究对象的多少倍?这个“多少倍”就是RR值

**OR值**：在回顾性研究(如case-control)中，研究对象是已经患病的“病例组”和未患病的“对照组”，研究者回顾性地调查病例组和对照组的暴露情况，因此无法计算发病率等指标。

想知道相对风险仍是我们最终的目的。

因此我们可以使用 OR值 来近似估计 RR值。

1. 当终点事件发生率较低时，OR值可以近似为RR值（$<15%$）
2. 当终点事件发生率较高时，OR会“夸大”RR值
   - OR值相对于RR值“更远离1”
   - 当RR值大于1时，OR大于RR($1<RR<OR$)
   - 当RR值小于1时，OR小于RR($OR<RR<1$)
   - 终点事件发生率越高时，OR越会overestimate

对于多列研究/RCT，可以报告OR值吗？

- 可以，但是不够准确/精准
- RR值对于效应值的估计更加准确
- RR值对于临床意义的解释更加明确
- Regression model中：对于结局是二分类变量的研究，logistic回归只能提供OR值，不能提供RR值(当结局发生率高时，应该使用log-binomial回归或者使用带有稳健方差估计的泊松回归，直接提供RR值)

### OR值的计算

语法：

```stata
cc var_case var_exp [if] [in] [weight] [,cc_options]
```

```stata
cci #a #b #c #d [,cci_options]
```

### 数据载入

In [13]:
%%stata
webuse ccxmpl,clear
list


. webuse ccxmpl,clear

. list

     +-----------------------+
     | case   exposed    pop |
     |-----------------------|
  1. |    1         1      4 |
  2. |    1         0    386 |
  3. |    0         1      4 |
  4. |    0         0   1250 |
     +-----------------------+

. 


In [14]:
%%stata
cc case exp [fweight = pop]

                                                         Proportion
                 |   Exposed   Unexposed  |      Total      exposed
-----------------+------------------------+------------------------
           Cases |         4         386  |        390       0.0103
        Controls |         4        1250  |       1254       0.0032
-----------------+------------------------+------------------------
           Total |         8        1636  |       1644       0.0049
                 |                        |
                 |      Point estimate    |    [95% conf. interval]
                 |------------------------+------------------------
      Odds ratio |         3.238342       |    .5997233    17.45614 (exact)
 Attr. frac. ex. |            .6912       |   -.6674356    .9427136 (exact)
 Attr. frac. pop |         .0070892       |
                 +-------------------------------------------------
                               chi2(1) =     3.07  Pr>chi2 = 0.0799


In [10]:
%%stata
cci 4 386 4 1250

                                                         Proportion
                 |   Exposed   Unexposed  |      Total      exposed
-----------------+------------------------+------------------------
           Cases |         4         386  |        390       0.0103
        Controls |         4        1250  |       1254       0.0032
-----------------+------------------------+------------------------
           Total |         8        1636  |       1644       0.0049
                 |                        |
                 |      Point estimate    |    [95% conf. interval]
                 |------------------------+------------------------
      Odds ratio |         3.238342       |    .5997233    17.45614 (exact)
 Attr. frac. ex. |            .6912       |   -.6674356    .9427136 (exact)
 Attr. frac. pop |         .0070892       |
                 +-------------------------------------------------
                               chi2(1) =     3.07  Pr>chi2 = 0.0799
